In [24]:
import numpy as np
from sklearn import datasets 
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import linear_model

In [511]:
iris = datasets.load_iris()
X = iris.data
scaledX = preprocessing.scale(X)
Y = iris.target

alphas = np.logspace(-4,4,20)
for alpha in alphas: 
    Xtrain, Xtest, Ytrain, Ytest = \
        model_selection.train_test_split(scaledX, Y, \
            test_size=0.1, shuffle=True)
    clf = linear_model.SGDClassifier(loss='squared_loss', \
        penalty='l2', alpha=alpha, max_iter=500000)
    clf.fit(Xtrain, Ytrain)
    print(clf.coef_)

[[-7.95933128e+10 -2.21513982e+10  3.04231536e+11 -6.24421759e+11]
 [-1.26260861e+12 -2.13409831e+11 -7.40302430e+11 -7.52879978e+11]
 [-2.61413022e+11 -4.99736744e+10  2.51661433e+11 -1.21919920e+11]]
[[-4.08072653e+11 -8.69032511e+10 -1.08279471e+10 -3.18979475e+10]
 [-5.25020868e+11 -7.86463542e+11 -6.05423469e+10 -3.58196004e+09]
 [-2.56939710e-01 -3.46344672e-01 -1.01170470e-01  7.13123333e-01]]
[[ 0.07119806 -0.01369847 -0.77617036 -0.01987003]
 [-0.13574989 -0.29874958  0.91271097 -0.589135  ]
 [-0.0153264   0.00712932  0.05255264  0.85172669]]
[[ 0.07533095  0.24420356 -0.64515538 -0.16168128]
 [ 0.23921436 -0.37445812  0.57509267 -0.65671331]
 [-0.27244102  0.18903434  0.33390226  0.69917742]]
[[ 0.63736454 -0.05608086 -2.46308173  1.01719475]
 [ 0.5033802  -0.61709501 -0.8584388   0.32996942]
 [-0.52356022  0.38366859  1.38865453 -0.08132491]]
[[  4.39156544  -1.73196586 -14.2211812    8.90524795]
 [  4.01469178  -2.26567887 -11.97245103   7.80098715]
 [ -3.94839963   1.96636

C:\Users\ychen\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


array([1.00000000e-04, 2.78255940e-04, 7.74263683e-04, 2.15443469e-03,
       5.99484250e-03, 1.66810054e-02, 4.64158883e-02, 1.29154967e-01,
       3.59381366e-01, 1.00000000e+00])